# Urban Climate Homework 1

In [1]:
import pandas as pd
import plotly.graph_objects as go
import calendar
import numpy as np

## Cleaning data

In [2]:
class data_explore: 

    """
    A class for exploring and cleaning data.

    Attributes:
        df (DataFrame): The input data frame.
        name (str): The name of the file.

    Methods:
        data_cleaning(): Cleans the data by dropping rows with -9999 values and saves the cleaned data to a CSV file.
    """

    def __init__(self, df, file_name) -> None:

        self.df = df
        self.name = file_name

        """
        Initializes the data_explore object.

        Args:
            df (DataFrame): The input data frame.
            file_name (str): The name of the file.
        """

    def data_celaning(self):

        """
        Cleans the data by changing rows with -9999 values to NaN values and saves the cleaned data to a CSV file.
        """

        df_columns = ["year", "month", "day", "Mean T", "Maximum T", "Minimum T"]

        self.df.columns = df_columns

        # print((self.df == -9999).any())

        for col in df_columns:
            # print("Column name = ", col)

            index_v = self.df[self.df[col] == -9999].index.tolist()

            df_clean = self.df.drop(self.df[self.df[col] == -9999].index)
            df_clean_nan = self.df.replace(-9999,np.NaN)
            

        # print((df_clean == -9999).any())

        df_clean.to_csv(self.name + "_Clean.csv")
        df_clean_nan.to_csv(self.name + "NaN_Clean.csv")

        return index_v, df_clean





#### Small index correction

In [3]:

df_urban = pd.read_csv("save_DMH2.csv")
df_rural = pd.read_csv("save_BWI2.csv")

index_values_rural, clean_rural_df = data_explore(df_rural, "rural").data_celaning()
index_values_urban, clean_urban_df= data_explore(df_urban, "urban").data_celaning()

# Since we only eliminate values from the Urban DataFrame, we need to eliminate that rows in the rural data.

df_clean = clean_rural_df.drop(index_values_urban)

df_clean.to_csv("rural_Clean.csv")


## Part 2

### UHI Intensity DataFrame


* Calculate the urban heat island intensity using daily mean, maximum, and minimum temperatures for each day and then average them to each month (no figure or value is needed). Then average the monthly results from 2000 to 2016. For example, you should average all the January results from 2000 to 2016 to obtain an averaged January urban heat island effect. Plot the averaged urban heat islands as a function of month (you can think of this as the averaged seasonal cycle of urban heat island). Use different colors or markers for daily mean, maximum, and minimum temperatures but plot them on the same figure.


In order to calculate Urban Heat Island Intensity we can use: 

$$ UHI_{\text{Intensity}} = \Delta T = T_{urban} - T_{rural}$$

In [4]:
def UHI_Intensity(urband_file_name, rurald_file_name, T_type):

    """
    Calculates the Urban Heat Island (UHI) intensity using temperature data from urban and rural areas.

    Args:
        urband_file_name (str): The file name of the urban temperature data.
        rurald_file_name (str): The file name of the rural temperature data.
        T_type (str): The type of temperature to calculate the UHI intensity for.

    Returns:
        None. The calculated UHI intensity values are saved to a CSV file.
    """

    df_urban = pd.read_csv(urband_file_name)
    df_rural = pd.read_csv(rurald_file_name)

    m = calendar.month_name[1:]

    mean_month_temp_rural = []
    mean_month_temp_urban = []

    UHI_df = pd.DataFrame()
    temp_df = pd.DataFrame()

    temp_df["Month"] = m
    UHI_df["Month"] = m

    years = 2000

    for ii in range(17):

        j = 1

        df_rural_year = df_rural.loc[df_rural['year'] == years]
        df_urban_year = df_urban.loc[df_urban['year'] == years]

        for jj in range(len(temp_df)):

            df_rural_month = df_rural_year.loc[df_rural_year['month'] == j]
            df_urban_month = df_urban_year.loc[df_urban_year['month'] == j]
            
            Mean_rural = df_rural_month.loc[:, T_type].mean()
            Mean_urban = df_urban_month.loc[:, T_type].mean()


            mean_month_temp_rural.append(Mean_rural)
            mean_month_temp_urban.append(Mean_urban)
            
            j = j + 1

        mean_month_temp_rural_C = [FahrenheitToCelsius(f) for f in mean_month_temp_rural]
        mean_month_temp_urban_C = [FahrenheitToCelsius(f) for f in mean_month_temp_urban]

        UHI_month_intensity_C = np.array(mean_month_temp_urban_C) - np.array(mean_month_temp_rural_C)
        
        temp_df[T_type + "_rural " + str(years)] = mean_month_temp_rural_C
        temp_df[T_type + "_urban " + str(years)] = mean_month_temp_urban_C
        UHI_df["UHI_intensity" + T_type + " " + str(years)] = UHI_month_intensity_C
        
        mean_month_temp_rural = []
        mean_month_temp_urban = []

        years = years + 1


    UHI_df['Mean'] = UHI_df.drop('Month', axis=1).apply(lambda x: x.mean(), axis=1)
    
    temp_df.to_csv(T_type + "_temperature_df.csv")
    UHI_df.to_csv("UHI_intensity" + T_type + ".csv")



In [16]:

def FahrenheitToCelsius(f):

    """
    Converts a temperature value in Fahrenheit to Celsius.

    Args:
        f (float): The temperature value in Fahrenheit.

    Returns:
        float: The temperature value converted to Celsius.
    """

    c = (f - 32) * 5 / 9

    return c



In [5]:

T_type = ["Mean T", "Maximum T", "Minimum T"]

for Temperature_type in T_type:

    UHI_Intensity("urban_Clean.csv", "rural_Clean.csv", Temperature_type)
        


In [6]:
df_mean = pd.read_csv("UHI_intensityMean T.csv")

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_mean.columns),
                line_color='rgb(8, 81, 156)', fill_color='rgb(8, 81, 156)', font=dict(color='white'),
                align='left'),
    cells=dict(values=df_mean.transpose().values.tolist(),
               align='left'))
])

fig.show()

### Plots

In [7]:
"""
This code reads the UHI intensity data from the CSV files and creates a line chart using the `plotly.graph_objects` module. 
The resulting chart displays the UHI intensity values for maximum, mean, and minimum temperature values across different months. 
The chart includes a title, x-axis label, and y-axis label.

To run this code and visualize the UHI intensity values, make sure you have the necessary libraries imported and the required CSV 
files available in the specified file paths.
"""

df_max = pd.read_csv("UHI_intensityMaximum T.csv")
df_mean = pd.read_csv("UHI_intensityMean T.csv")
df_min = pd.read_csv("UHI_intensityMinimum T.csv")


fig = go.Figure()

fig.add_trace(go.Scatter(x=df_max["Month"], y=df_max["Mean"],
                    mode='lines+markers',
                    name='Max'))
fig.add_trace(go.Scatter(x=df_mean["Month"], y=df_mean["Mean"],
                    mode='lines+markers',
                    name='Mean'))
fig.add_trace(go.Scatter(x=df_min["Month"], y=df_min["Mean"],
                    mode='lines+markers',
                    name='Min'))


fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="All",
                     method="update",
                     args=[{"visible": [True, True, True]},
                           {"title": "UHI Intensity for Max, Min and Mean Values of Temperature",
                            "annotations": []}]),
                dict(label="Min",
                     method="update",
                     args=[{"visible": [False, False, True]},
                           {"title": "UHI Intensity for Minimum Values of Temperature",
                            "annotations": []}]),
                dict(label="Max",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "UHI Intensity for Maximum Values of Temperature",
                            "annotations": []}]),
                dict(label="Mean",
                     method="update",
                     args=[{"visible": [False, True, False]},
                           {"title": "UHI Intensity for Mean Values of Temperature",
                            "annotations": []}]),
            ]),
        )
    ])

fig.update_layout(title='UHI Intensity for Max, Min and Mean Values of Temperature',
                   xaxis_title='Month',
                   yaxis_title='UHI Intensity')

fig.show()

### Discussion: 

a) which season has the strongest urban heat island effect and what are the possible causes.

* Answer: 

b) which temperature (mean, maximum, or minimum) yields the strongest urban heat island effect. What are the possible causes?

* Answer: 



## Part 2

* Calculate the urban heat island intensity using daily mean, maximum, and minimum temperatures for each day (no figure or value is needed). Average the daily urban heat island intensities over the summer season (including June, July, August) in the period of 2000-2016 and report the values. Report the values for daily mean, maximum, and minimum temperature separately. Here you should not take the simple average of June, July, and August results you obtain in 1 because June/July/August have different numbers of days, but you also show not report values for June, July, August separately. All I want is the values for the summer season, not for individual months in the summer season.


In [23]:

def UHI_Summer_Intensity(urband_file_name, rurald_file_name, T_type):

    df_urban = pd.read_csv(urband_file_name)
    df_rural = pd.read_csv(rurald_file_name)

    years = 2000

    UHI_summer = pd.DataFrame()
    urban_T_summer = pd.DataFrame()
    rural_T_summer = pd.DataFrame()

    rural_list = []
    urban_list = []

    for ii in range(17):

        df_rural_year = df_rural.loc[df_rural['year'] == years]
        df_urban_year = df_urban.loc[df_urban['year'] == years]

        j = 6

        for jj in range(3):

            df_rural_month = df_rural_year.loc[df_rural_year['month'] == j]
            list1 = df_rural_month.loc[:, T_type].tolist()
            rural_list = list1 + rural_list
            df_urban_month = df_urban_year.loc[df_urban_year['month'] == j]
            list2 = df_urban_month.loc[:, T_type].tolist()
            urban_list = list2 + urban_list
                
            j = j + 1


        summer_temp_rural_C = [FahrenheitToCelsius(f) for f in rural_list]
        summer_temp_urban_C = [FahrenheitToCelsius(f) for f in urban_list]
        UHI_summer_intensity_C = np.array(summer_temp_urban_C) - np.array(summer_temp_rural_C)

        urban_T_summer[T_type + " " + str(years)] = summer_temp_urban_C
        rural_T_summer[T_type + " " + str(years)] = summer_temp_rural_C
        UHI_summer["UHI" + T_type + " " + str(years)] = UHI_summer_intensity_C

        rural_list = []
        urban_list = []

        years = years + 1
        
    urban_T_summer['Mean'] = urban_T_summer.apply(lambda x: x.mean(), axis=1)
    rural_T_summer['Mean'] = rural_T_summer.apply(lambda x: x.mean(), axis=1)
    UHI_summer['Mean'] = UHI_summer.apply(lambda x: x.mean(), axis=1)

    UHI_summer.to_csv("UHI_summer " + T_type + ".csv")
    urban_T_summer.to_csv("Urban_summer " + T_type + ".csv")
    rural_T_summer.to_csv("Rural_summer " + T_type + ".csv")


In [24]:
T_type = ["Mean T", "Maximum T", "Minimum T"]

for Temperature_type in T_type:

    UHI_Summer_Intensity("urbanNaN_Clean.csv", "ruralNaN_Clean.csv", Temperature_type)
    


In [45]:
df_mean1 = pd.read_csv("UHI_summer Mean T.csv")
df_mean = list(df_mean1.loc[:,'Mean'])

fig = go.Figure(data=[go.Table(
    header=dict(values=['Mean'],
                line_color='rgb(8, 81, 156)', fill_color='rgb(8, 81, 156)', font=dict(color='white'),
                align='left'),
    cells=dict(values=[df_mean1.Mean],
               align='left'))
])
fig.show()

* Focus on daily results in the summer (June, July, August) seasons in the period of 2000-2016 (you should have 17 years x 92 summer days per year = 1564 summer days). Plot the daily urban heat island intensity (use different subplots or subfigures for daily mean, maximum, and minimum temperatures) as a function of the daily mean temperature of the rural site and fit a linear relation between the two. How does the urban heat island intensity change with the mean temperature of the rural site?


In [ ]:
df_max = pd.read_csv("UHI_summer Maximum T.csv")
df_mean = pd.read_csv("UHI_summer Mean T.csv")
df_min = pd.read_csv("UHI_summer Minimum T.csv")


fig = go.Figure()

fig.add_trace(go.Scatter(x=df_max["Month"], y=df_max["Mean"],
                    mode='lines+markers',
                    name='Max'))
fig.add_trace(go.Scatter(x=df_mean["Month"], y=df_mean["Mean"],
                    mode='lines+markers',
                    name='Mean'))
fig.add_trace(go.Scatter(x=df_min["Month"], y=df_min["Mean"],
                    mode='lines+markers',
                    name='Min'))


fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="All",
                     method="update",
                     args=[{"visible": [True, True, True]},
                           {"title": "UHI Intensity for Max, Min and Mean Values of Temperature",
                            "annotations": []}]),
                dict(label="Min",
                     method="update",
                     args=[{"visible": [False, False, True]},
                           {"title": "UHI Intensity for Minimum Values of Temperature",
                            "annotations": []}]),
                dict(label="Max",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "UHI Intensity for Maximum Values of Temperature",
                            "annotations": []}]),
                dict(label="Mean",
                     method="update",
                     args=[{"visible": [False, True, False]},
                           {"title": "UHI Intensity for Mean Values of Temperature",
                            "annotations": []}]),
            ]),
        )
    ])

fig.update_layout(title='UHI Intensity for Max, Min and Mean Values of Temperature',
                   xaxis_title='Month',
                   yaxis_title='UHI Intensity')

fig.show()